In [16]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt; plt.style.use('ggplot')
from scipy import stats

In [17]:
data = pd.read_csv('/Users/willfitzhugh/Desktop/Coding/Supply-Chain-Analysis/Data/Interim/Ready_For_Fraud_Features.csv')

In [5]:
#feature engineering to deal with catagorical variables with high cardinality:
#flag cities and states with high fraud rate, and those with no recorded fraud
#same with product names
#cols to engineer: ProductCategory, Customer State, Customer City, ProductDepartment, Order City, Order Country, Order region,
#Order state, Product Name

#cols to remove: Customer Id, Latitude, Longitude, Order date, Ship date, Order Id, Order status, Customer Name,
#order year, real ship days, delivery status, late_delivery_risk, order Status, order year and probably one or more 
#of [OrderProfit, OrderSales, Order Item Discount, Order Item Discount Rate,Order Item Profit Ratio, product price]

#cols to one hot encode: PaymentType, Customer Country, Customer Segment, Market, shipping mode, Order DOW

In [18]:
dums = pd.get_dummies(data[['PaymentType', 'Customer Country', 'Customer Segment', 'Market', 'Shipping Mode',
                            'OrderDOW']])

data = pd.concat([data,dums], axis=1).drop(columns=['PaymentType', 'Customer Country', 'Customer Segment', 'Market',
                                             'Shipping Mode','OrderDOW'])

In [19]:
#have different approach with columns with high cardinality. order city has much higher cardinality than order region
#and should be binned differently
'''
want to keep bins from over fitting to highly cardinal data. I want to avoid marking a city as extremely high risk 
just because it has one fraud order, and zero non-fraud orders. I still want to mark it has higher risk, but marking
it as garaunteed fraud might cause data leakage and over fitting.
'''
bin_cols = ['ProductCategory', 'Customer State', 'Customer City', 'ProductDepartment', 'Order City', 'Order Country',
            'Order Region', 'Order State', 'Product Name']

for i in bin_cols: 

    grouped = data.groupby(i).mean()[['IsFraud']].sort_values('IsFraud', ascending=False).reset_index()
    binned = stats.binned_statistic(x = grouped['IsFraud'], values = grouped['IsFraud'], bins = [0, .001, .015, .02, .03, .045, .2, 1])

    grouped['bin'] = binned[2]
    
    bin_map = grouped[[i,'bin']].set_index(i).to_dict()['bin']
    
    data[i] = data[i].map( bin_map )
    
    
'''
This mapping strategy/ target encoding in general presents a problem. I may not have data on an order city for a 
new order, so my model will not be able to handle the order. I could create another model, one that doesn't depend
on order city and use that model if the input order has an order city that didn't appear in the train dataset.
'''

"\nThis mapping strategy/ target encoding in general presents a problem. I may not have data on an order city for a \nnew order, so my model will not be able to handle the order. I could create another model, one that doesn't depend\non order city and use that model if the input order has an order city that didn't appear in the train dataset.\n"

In [20]:
data = data.drop(columns = ['Customer Id','Latitude','Longitude','OrderDate','ShipDate','Order Id','Order Status', 
                  'CustomerName', 'OrderYear', 'RealShippingDays', 'Delivery Status', 'Late_delivery_risk',
                  'Order Status','OrderHour','OrderMonth','Order Item Discount','OrderProfit','Order Item Quantity'])


In [21]:
data

,ScheduledShippingDays,OrderSales,ProductCategory,Customer City,Customer State,ProductDepartment,Order City,Order Country,Order Item Discount Rate,Order Item Profit Ratio,Order Region,Order State,Product Name,Product Price,IsFraud,PaymentType_CASH,PaymentType_DEBIT,PaymentType_PAYMENT,PaymentType_TRANSFER,Customer Country_EE. UU.,Customer Country_Puerto Rico,Customer Segment_Consumer,Customer Segment_Corporate,Customer Segment_Home Office,Market_Africa,Market_Europe,Market_LATAM,Market_Pacific Asia,Market_USCA,Shipping Mode_First Class,Shipping Mode_Same Day,Shipping Mode_Second Class,Shipping Mode_Standard Class,OrderDOW_Friday,OrderDOW_Monday,OrderDOW_Saturday,OrderDOW_Sunday,OrderDOW_Thursday,OrderDOW_Tuesday,OrderDOW_Wednesday
0,2,115.180000,4,2,4,4,1,4,0.04,-0.27,3,5,4,59.990002,False,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
1,2,79.180000,4,4,4,4,1,3,0.01,-1.55,4,1,4,39.990002,False,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,1,0
2,2,96.000000,4,4,4,4,1,4,0.04,0.35,4,2,4,50.000000,False,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,0,0,0,1
3,2,75.980003,4,4,4,4,1,3,0.05,0.33,4,3,4,39.990002,False,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,0
4,2,91.000000,4,4,4,4,1,1,0.09,0.10,4,1,4,50.000000,False,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,0,1,0,0,1,0,0,0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
173970,4,399.980011,4,3,3,4,6,4,0.00,0.10,4,6,4,399.980011,False,1,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0
173971,2,395.980011,4,5,4,4,1,3,0.01,-1.55,4,1,4,399.980011,False,0,1,0,0,1,0,0,1,0,0,0,0,1,0,0,0,1,0,0,0,1,0,0,0,0
173972,4,391.980011,4,2,4,4,2,4,0.02,0.36,4,2,4,399.980011,False,0,0,0,1,1,0,0,1,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0
173973,4,387.980011,4,4,4,4,2,4,0.03,0.48,4,2,4,399.980011,False,0,0,1,0,0,1,1,0,0,0,0,0,1,0,0,0,0,1,1,0,0,0,0,0,0


In [9]:
data.IsFraud = data.IsFraud.map({True:1,False:0})

In [10]:
data.to_csv('/Users/willfitzhugh/Desktop/Coding/Supply-Chain-Analysis/Data/Model/FraudData_1.0.csv', index=False)